# Featurestore - Churn Demo
## Data Injestion
Christos Aniftos

In [ ]:
from google.api_core import operations_v1
from google.cloud.aiplatform_v1beta1.types import io as io_pb2
from google.cloud.aiplatform_v1beta1.types.feature import Feature
from google.cloud.aiplatform_v1beta1 import FeaturestoreServiceClient
from google.cloud.aiplatform_v1beta1 import FeaturestoreOnlineServingServiceClient
from google.cloud.aiplatform_v1beta1.types import entity_type as entity_type_pb2
from google.cloud.aiplatform_v1beta1.types import featurestore as featurestore_pb2
from google.cloud.aiplatform_v1beta1.types import feature_selector as feature_selector_pb2
from google.cloud.aiplatform_v1beta1.types import featurestore_service as featurestore_service_pb2
from google.cloud.aiplatform_v1beta1.types import featurestore_online_service as featurestore_online_service_pb2

In [ ]:
PROJECT_ID=!gcloud config get-value project # returns default project id 
PROJECT_ID=PROJECT_ID[0]

LOCATION = 'europe-west4'
API_ENDPOINT = LOCATION+"-aiplatform.googleapis.com" 
FEATURESTORE_ID = "telco"
ENTITY="phone"

## Define clients for FS admin and data management

In [ ]:
# Create admin_client for CRUD 
admin_client = FeaturestoreServiceClient(
    client_options={"api_endpoint": API_ENDPOINT})

In [ ]:
LOC_PATH = admin_client.common_location_path(PROJECT_ID, LOCATION)
FS_PATH = admin_client.featurestore_path(PROJECT_ID, LOCATION, FEATURESTORE_ID)
ENTITY_PATH = admin_client.entity_type_path(PROJECT_ID, LOCATION, FEATURESTORE_ID, ENTITY)
FEATURE_PATH = admin_client.feature_path(PROJECT_ID, LOCATION, FEATURESTORE_ID, ENTITY, '{}')

print("Location: \t", LOC_PATH)
print("Feature Store: \t", FS_PATH)
print("Entity: \t", ENTITY_PATH)
print("Feature: \t",FEATURE_PATH)

## Injest

#### prepare aggregated data to injest- this creates a temporary bq table with the features

In [ ]:
from google.cloud import bigquery

# Construct a BigQuery client object.
client = bigquery.Client()

query = """
CREATE OR REPLACE TABLE `{project_id}.telco.phone_features_tmp` 
AS SELECT CONCAT(brand," - ",model) as phoneID, *, CURRENT_TIMESTAMP() as update_time  
FROM `{project_id}.telco.phone` 
""".format(project_id=PROJECT_ID)
query_job = client.query(query)  # Make an API request.

In [ ]:
query_job.result()

### Supported injest modes

avro_source (google.cloud.aiplatform_v1beta1.types.AvroSource):

bigquery_source (google.cloud.aiplatform_v1beta1.types.BigQuerySource):

csv_source (google.cloud.aiplatform_v1beta1.types.CsvSource):

In [ ]:
ENTITY_PATH

### Injest from temp table to feature store

In [ ]:
import_request = featurestore_service_pb2.ImportFeatureValuesRequest(
    entity_type=ENTITY_PATH,
    bigquery_source=io_pb2.BigQuerySource(
        input_uri="bq://{project_id}.telco.phone_features_tmp".format(project_id=PROJECT_ID)
    ),
    feature_specs=[
        featurestore_service_pb2.ImportFeatureValuesRequest.FeatureSpec(
            id="approx_price_euro", source_field="approx_price_EUR"),
        featurestore_service_pb2.ImportFeatureValuesRequest.FeatureSpec(
            id="brand", source_field="brand"),
        featurestore_service_pb2.ImportFeatureValuesRequest.FeatureSpec(
            id="model", source_field="model"),
    ],
    entity_id_field="phoneID",
    feature_time_field="update_time",
    worker_count=1)

In [ ]:
%%time
admin_client.import_feature_values(import_request).result()